# 数据处理
在pytorch中，主要有两个工具包涉及到数据处理：
1. torch.utils.data
2. torchvision
![title](./img/dataprocess.png)



# 数据可视化
pytorch从1.2版本开始支持tensorboard，即使用pip install tensorboard安装tensorboard之后，可以通过  
在命令行窗口输入：**tensorboard --logdir=logs --port 6006**  
**import torch.utils.tensorboard import SummaryWrite**进行导入。  
**注：在pytorch1.2版本之前都需要安装tensorboardX**
***
![title](./img/summarywriter.png)


In [27]:
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

## 使用tensorboard可视化神经网络

In [48]:
 # 构建神经网络
import torch.nn as nn
import torch.nn.functional as F
# 假设网络的输入维度为CxHxW: 1x28x28
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # 默认stride=1
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        self.bn = nn.BatchNorm2d(20) # 20为输入的通道数
        
    def forward(self, x):
        x = self.conv1(x) # 10x24x24
        x = F.max_pool2d(x, 2) # 默认stride=kernel_size, 10x12x12
        x = F.relu(x)
        x = self.conv2(x) # 20x8x8
        x = F.max_pool2d(x, 2) # 20x4x4
        x = self.bn(x)
        x = x.view(-1, 320)
        x = self.fc1(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

# 定义输入
inputs = torch.randn(32, 1, 28, 28)
# print(inputs.shape)
# 定义模型
model = Net()
with SummaryWriter(log_dir="./logs", comment="Net") as w:
    w.add_graph(model, inputs)
        

## 可视化损失值

In [57]:
import numpy as np
np.random.seed(100)
dtype = torch.FloatTensor
writer = SummaryWriter(log_dir="./logs/loss/", comment="loss")
x = np.linspace(-1, 1, 100).reshape(-1, 1)
y = 3*np.power(x, 2) + 2 + 0.2*np.random.randn(100,1)

model = nn.Linear(1, 1)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

for epoch in range(80):
    inputs = torch.from_numpy(x).type(dtype)
    outputs = torch.from_numpy(y).type(dtype)
    
    preds = model(inputs)
    loss = criterion(preds, outputs)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # 保存每次迭代的损失
    writer.add_scalar("loss", loss, epoch)
writer.close()